# <center> TP DE SYSTEME DE RECOMMANDATION

In [2]:
import pandas as pd
bases=["movies_metadata.csv","ratings_small.csv"]
path="/Users/johankakou/Library/CloudStorage/OneDrive-ENSEA/Vie pro/ISE 3/Cours 2022-2023/Option data science/Machine learning avancé/Projet Recommender system -- KAKOU Johan/Application web/data/"
data={i.replace(".csv", ""):pd.read_csv(path+i) for i in bases}

/var/folders/df/rtd8q41s4lg7pk5mj05920sw0000gn/T/ipykernel_38758/1468756421.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data={i.replace(".csv", ""):pd.read_csv(path+i) for i in bases}


In [3]:
! pip install faker

In [4]:
#Base utilisateurs
import pandas as pd
from faker import Faker
import random
import hashlib
# Créer une instance de Faker
fake = Faker("fr_FR")
# Définir la graine aléatoire
random.seed(1234)
utilisateurs=pd.DataFrame(data["ratings_small"]["userId"].unique())

In [5]:
utilisateurs["nom_utilisateur"]=utilisateurs[0].apply(lambda x: fake.last_name()+" "+fake.first_name())
utilisateurs["mdp_crypte"]=utilisateurs[0].apply(lambda x:hashlib.sha256(fake.word().encode()).hexdigest())
utilisateurs["genre"]=utilisateurs[0].apply(lambda x:fake.random_element(elements=('Homme', 'Femme')))
utilisateurs["ville"]=utilisateurs[0].apply(lambda x:fake.city())
utilisateurs["mail"]=utilisateurs[0].apply(lambda x:fake.email())

In [6]:
utilisateurs= utilisateurs.rename(columns={0: 'id'})

In [7]:
utilisateurs.head()

,id,nom_utilisateur,mdp_crypte,genre,ville,mail
0,1,Carre Édith,386f8dd68b2c1bdc54c0bc5717bb3a69f5339c2f22bac5...,Homme,Allard,ferrandwilliam@example.org
1,2,Pascal Olivier,6f90ae80068cdc7054d6e9e9ab930a9d21a9ac654971fa...,Homme,Guilbert-sur-Nicolas,qdelannoy@example.net
2,3,Gomez Laurent,c8ad48290b8edbb4ef18afbb59257069b48193c5452ef3...,Homme,Berger,hoaraudaniel@example.com
3,4,Bodin Monique,5b81c7c3bb698fbfea7fc79fe3f11cb5fde0861b94fffa...,Homme,Faivre,margaretgros@example.com
4,5,Paris Bernard,53b95a34f534b719dedb60ef39d15a1367c77743978708...,Homme,Meunier-sur-Sanchez,sophiebesnard@example.org


In [8]:
#Base films
films=data["movies_metadata"].loc[:,["id","title","tagline","genres","overview","release_date","runtime","vote_average","vote_count"]]
films.rename(columns={"title":"titre","overview":"description","release_date":"date_sortie","runtime":"duree","vote_average":"vote_moyen","vote_count":"nbre_votes"},inplace=True)

In [19]:
#!pip install --upgrade sqlalchemy
import datetime
from JoStreamIvoire import app,db
from JoStreamIvoire.models import Utilisateur,Film
def inserer_utilisateur(user):
    user_to_create = Utilisateur(nom_utilisateur=user["nom_utilisateur"],
                                     mail=user["mail"],
                                     password=user["mdp_crypte"],
                                     gender=user["genre"],
                                     ville=user["ville"]
                                    )
    db.session.add(user_to_create)
    db.session.commit()
def inserer_film(film):
    if film["tagline"]:
        film["tagline"]="r.a.s"
    film_to_create = Film(  titre=film["titre"],
                            tagline=film["tagline"],
                            genres=film["genres"],
                            description=film["description"],
                            duree=film["duree"],
                            nbre_votes=film["nbre_votes"],
                            vote_moyen=film["vote_moyen"]
                        )
    db.session.add(film_to_create)
    db.session.commit()  

In [16]:
db.session.rollback()

In [20]:
Film.query.all()

[]

In [22]:
with app.app_context():
    for id_film in films.head(10000).index:
        try :
            inserer_film(films.iloc[id_film,:])
        except:
            pass
    for id_utilisateur in utilisateurs.index:
        try :
            inserer_utilisateur(utilisateurs.iloc[id_utilisateur,:])  
        except:
            pass

/var/folders/df/rtd8q41s4lg7pk5mj05920sw0000gn/T/ipykernel_38758/1968835790.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  film["tagline"]="r.a.s"


In [51]:
with app.app_context():
    for id_film in films.head(10000).index:
        inserer_film(films.iloc[id_film,:])
    for id_utilisateur in utilisateurs.index:
        inserer_utilisateur(utilisateurs.iloc[id_utilisateur,:])    

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/base.py", line 1670, in _execute_context
    context = constructor(
  File "/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 1052, in _init_compiled
    param = [
  File "/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 1053, in <listcomp>
    processors[key](compiled_params[key])
  File "/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/sqlalchemy/sql/sqltypes.py", line 2069, in process
    value = epoch + value
TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'float'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, sel

### a) Chargement du jeu de donnees

In [5]:
data["movies_metadata"]
mov_met=data["movies_metadata"]
mov_met.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [87]:
#def probleme_conversion(x): 
    try :
        int(x)
        return True 
    except ValueError:
        return False
#films.drop(films[films["id"].apply(probleme_conversion)].index,inplace=True)

IndentationError: unexpected indent (3225853541.py, line 2)

In [ ]:
#films.loc[:,"duree"].astype(float)
#films.loc[:,"vote_moyen"].astype(float)
#films["id"].astype(int)
#films["nbre_vote"].astype(int)

In [ ]:
films.shape

(45466, 9)

In [11]:
#!pip install --upgrade sqlalchemy
import pandas as pd
from sqlalchemy import create_engine

# Créer une connexion à votre base de données
engine = create_engine('sqlite:///JoStreamIvoire.db')

# Exporter le DataFrame dans une table SQL
table_name = 'Film'  # Remplacez par le nom de votre table SQL
films.head(10).to_sql(table_name, con=engine, if_exists='replace', index=False)

# Fermer la connexion à la base de données
engine.dispose()

In [12]:
#Base votes
votes=data["ratings_small"]
table_name = 'votes'  # Remplacez par le nom de votre table SQL
votes.head(10).to_sql(table_name, con=engine, if_exists='replace', index=False)

# Fermer la connexion à la base de données
engine.dispose()

L’une des mesures les plus élémentaires aux quelles vous pouvez penser est le classement pour décider quels 250 meilleurs films sont basés sur leurs notes respectives.

Cependant, l’utilisation d’une évaluation comme métrique comporte quelques mises en garde :

<p><li>D’une part, cela ne prend pas en considération la popularité d’un film. Par conséquent, un film avec une note de 9 sur 10 votants sera considéré comme « meilleur » qu’un film avec une note de 8,9 de 10 000 votants.

<p>Par exemple, imaginez que vous voulez commander de la nourriture chinoise, vous avez quelques options, un restaurant a une note de 5 étoiles par seulement 5 personnes tandis que l’autre restaurant a 4,5 notes par 1000 personnes. Quel restaurant préféreriez-vous? Le deuxième, n’est-ce pas?

<p>Bien sûr, il pourrait y avoir une exception que le premier restaurant a ouvert il y a quelques jours; Ainsi, moins de gens ont voté pour alors qu’au contraire, le deuxième restaurant est opérationnel depuis un an.

<p><li>Dans le même ordre d’idées, cette mesure aura également tendance à favoriser les films avec un plus petit nombre d’électeurs avec des notes biaisées et / ou extrêmement élevées. Au fur et à mesure que le nombre de votants augmente, la note d’un film se régule et se rapproche d’une valeur qui reflète la qualité du film et donne à l’utilisateur une bien meilleure idée du film qu’il devrait choisir. Bien qu’il soit difficile de discerner la qualité d’un film avec extrêmement peu d’électeurs, vous devrez peut-être tenir compte de sources externes pour conclure.

En tenant compte de ces lacunes, vous devez proposer une note pondérée qui tient compte de la note moyenne et du nombre de votes qu’elle a accumulés. Un tel système garantira qu’un film avec une note de 9 sur 100 000 votants obtient un score (beaucoup) plus élevé qu’un film avec la même note mais seulement quelques centaines d’électeurs.

Puisque vous essayez de construire un clone du Top 250 d’IMDB, utilisons sa formule de notation pondérée comme métrique / score. Mathématiquement, il est représenté comme suit:

## Weighted Rating

$$ (\bf WR) = \left({{\bf v} \over {\bf v} + {\bf m}} \cdot R\right) + \left({{\bf m} \over {\bf v} + {\bf m}} \cdot C\right)$$

Dans l’équation ci-dessus,

<li>v est le nombre de votes pour le film;

<li>m est le nombre minimum de votes requis pour figurer dans le tableau;

<li>R est la note moyenne du film;

<li>C est le vote moyen pour l’ensemble du rapport.

Vous disposez déjà des valeurs v et R pour chaque film du jeu de données. Il est également possible de calculer directement C à partir de ces données.(vote_count)(vote_average)

La détermination d’une valeur appropriée pour est un hyperparamètre que vous pouvez choisir en conséquence puisqu’il n’y a pas de valeur correcte pour . Vous pouvez le considérer comme un filtre négatif préliminaire qui supprimera simplement les films qui ont un nombre de votes inférieur à un certain seuil . La sélectivité de votre filtre est à votre discrétion.

Vous utiliserez cutoff comme 90e percentile. En d’autres termes, pour qu’un film figure dans les charts, il doit avoir plus de votes qu’au moins 90% des films de la liste. (D’un autre côté, si vous aviez choisi le 75e centile, vous auriez considéré les 25% des films les plus performants en termes de nombre de votes recueillis. Au fur et à mesure que le percentile diminue, le nombre de films considérés augmente).

Dans un premier temps, calculons la valeur de C, la note moyenne de tous les films à l’aide de la fonction pandas:.mean()


## 1_a Calculate mean of vote average column

In [4]:
C=mov_met["vote_average"].mean()
C

5.618207215134185

Ensuite, calculons le nombre de votes, m, reçus par un film dans le 90e centile. La bibliothèque rend cette tâche extrêmement triviale en utilisant la méthode des pandas:pandas.quantile()

## 1_b Calculate the minimum number of votes required to be in the chart, m

In [5]:
m=mov_met["vote_count"].quantile(0.9)
m

160.0

Puisque maintenant vous avez le vous pouvez simplement utiliser une condition supérieure à égale pour filtrer les films ayant plus de 160 votes comptés:m

Vous pouvez utiliser la méthode pour vous assurer que le nouveau DataFrame créé est indépendant de votre DataFrame de métadonnées d’origine. En d’autres termes, les modifications apportées au DataFrame q_movies n’affecteront pas le cadre de données de métadonnées d’origine..copy()q_movies

## 1_c Filter out all qualified movies into a new DataFrame

In [6]:
q_movies=mov_met[mov_met["vote_count"]>=m]
q_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0


L’étape suivante est la plus importante consiste à calculer la note pondérée pour chaque film qualifié. Pour ce faire, vous allez :

<li>Définir une fonction, weighted_rating();
<li>Puisque vous avez déjà calculé et que vous les passerez simplement comme argument à la fonction;mC
<li>Ensuite, vous sélectionnerez les colonnes (v) et (R) dans le bloc de données ;vote_countvote_averageq_movies
<li>Enfin, vous calculerez la moyenne pondérée et retournerez le résultat.


Vous allez définir une nouvelle fonctionnalité , dont vous calculerez la valeur en appliquant cette fonction à votre DataFrame de films qualifiés :score

## 1_d Function that computes the weighted rating of each movie

In [7]:
R=q_movies["vote_count"]
v=q_movies["vote_average"]

## 1_e Define a new feature 'score' and calculate its value with `weighted_rating()

In [8]:
q_movies["score"]=(v*R+m*C)/(v+m)

/var/folders/df/rtd8q41s4lg7pk5mj05920sw0000gn/T/ipykernel_14512/3698809866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_movies["score"]=(v*R+m*C)/(v+m)


In [9]:
q_movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,253.991730
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,105.144477
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,11.376060


Enfin, trions le DataFrame par ordre décroissant en fonction de la colonne des fonctionnalités et sortons le titre, le nombre de votes, la moyenne des votes et la note pondérée (score) des 20 meilleurs films.score

## 1_f Sort movies based on score calculated above

In [10]:
q_movies.sort_values(by="score",ascending=False,inplace=True)

/var/folders/df/rtd8q41s4lg7pk5mj05920sw0000gn/T/ipykernel_14512/2406112185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_movies.sort_values(by="score",ascending=False,inplace=True)


## 1_g Print the top 15 movies

In [11]:
top_15=q_movies[["title","vote_count","vote_average","score"]].head(15)
top_15

,title,vote_count,vote_average,score
15480,Inception,14075.0,8.1,683.559864
12481,The Dark Knight,12269.0,8.3,610.407684
22879,Interstellar,11187.0,8.1,544.399840
17818,The Avengers,12000.0,7.4,535.835801
14551,Avatar,12114.0,7.2,527.031777
26564,Deadpool,11444.0,7.4,511.257546
20051,Django Unchained,10297.0,7.8,484.001866
2843,Fight Club,9678.0,8.3,482.628123
23753,Guardians of the Galaxy,10014.0,7.9,476.530751
292,Pulp Fiction,8670.0,8.3,432.916893


# 2)Recommandateur basé sur le contenu

### Recommandateur basé sur la description de l’intrigue

 vous allez apprendre à créer un système qui recommande des films similaires à un film particulier. Pour ce faire, vous calculerez les scores de similarité par paires pour tous les films en fonction de leurs descriptions d’intrigue et recommanderez des films en fonction de ce seuil de score de similarité.cosinus

La description de la parcelle est disponible en tant que fonctionnalité de votre jeu de données. Examinons les intrigues de quelques films:overviewmetadata

## 2_1 Print plot overviews of the first 5 movies.

In [12]:
print(f"Les colonnes disponibles sont: {list(data.keys())}")

Les colonnes disponibles sont: ['credits', 'links', 'movies_metadata', 'ratings_small', 'keywords', 'links_small', 'ratings']


In [13]:
print(f'Un exemple de revue du film {data["movies_metadata"]["title"][0]} est:\n\n{data["movies_metadata"]["overview"][:5]}')

Un exemple de revue du film Toy Story est:

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object


Le problème est un problème de traitement du langage naturel. Par conséquent, vous devez extraire certaines caractéristiques des données textuelles ci-dessus avant de pouvoir calculer la similitude et / ou la dissemblance entre elles. Pour le dire simplement, il n’est pas possible de calculer la similitude entre deux aperçus dans leurs formes brutes. Pour ce faire, vous devez calculer les vecteurs de mots de chaque vue d’ensemble ou document, comme on l’appellera à partir de maintenant.

Comme leur nom l’indique, les vecteurs de mots sont une représentation vectorisée de mots dans un document. Les vecteurs portent avec eux une signification sémantique. Par exemple, l’homme et le roi auront des représentations vectorielles proches l’un de l’autre tandis que l’homme et la femme auront une représentation éloignée l’un de l’autre.

Vous calculerez des vecteurs (TF-IDF) pour chaque document. Cela vous donnera une matrice où chaque colonne représente un mot dans le vocabulaire de la vue d’ensemble (tous les mots qui apparaissent dans au moins un document), et chaque colonne représente un film, comme précédemment.Term Frequency-Inverse Document Frequency

Dans son essence, le score TF-IDF est la fréquence d’un mot apparaissant dans un document, pondérée en fonction du nombre de documents dans lesquels il apparaît. Ceci est fait pour réduire l’importance des mots qui apparaissent fréquemment dans les aperçus de l’intrigue et, par conséquent, leur importance dans le calcul du score de similarité final.

Heureusement, scikit-learn vous donne une classe intégrée qui produit la matrice TF-IDF en quelques lignes.TfIdfVectorizer

<li>Importer le module Tfidf à l’aide de scikit-learn ;
<li>Supprimez les mots vides comme « le », « un », etc. puisqu’ils ne donnent aucune information utile sur le sujet;
<li>Remplacez les valeurs sans nombre par une chaîne vide ;
<li>Enfin, construisez la matrice TF-IDF sur les données.

In [14]:
# Pour la vectorisation
from sklearn.feature_extraction.text import TfidfVectorizer 

In [15]:
# Pour le pré processing
from unidecode import unidecode
import re
from nltk.stem import SnowballStemmer

# Pour la dataviz
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Les bigrammes
from collections import Counter
from nltk.util import ngrams

# Pour la vectorisation
from sklearn.feature_extraction.text import TfidfVectorizer 

# Pour la modélisation
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score

from sklearn.naive_bayes import GaussianNB

In [16]:
overview=q_movies.loc[:,["overview","title","score"]]
overview.head()

,overview,title,score
15480,"Cobb, a skilled thief who commits corporate es...",Inception,683.559864
12481,Batman raises the stakes in his war on crime. ...,The Dark Knight,610.407684
22879,Interstellar chronicles the adventures of a gr...,Interstellar,544.399840
17818,When an unexpected enemy emerges and threatens...,The Avengers,535.835801
14551,"In the 22nd century, a paraplegic Marine is di...",Avatar,527.031777


In [17]:
# Définition de la liste de stop words considérés (celle de spacy + lettres)
stopWords = pd.read_csv("NLTK's list of english stopwords.csv",header=None).loc[:,0].tolist()
# Création du stemmer
stopWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [18]:
stemmer = SnowballStemmer("english")

## Nettoyage de la base

In [19]:
# Type
overview.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4555 entries, 15480 to 9651
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   overview  4550 non-null   object 
 1   title     4555 non-null   object 
 2   score     4555 non-null   float64
dtypes: float64(1), object(2)
memory usage: 142.3+ KB


In [20]:
# zoom sur les valeurs manquante
overview.isnull().sum()

overview    5
title       0
score       0
dtype: int64

In [21]:
# Création d'une fonction pour supprimer les stopWords
def no_stop_word(string, stopWords):

    """
    Supprime les stop words d'un texte.

    Paramètres
    ----------

    string : chaine de caractère.

    stopWords : liste de mots à exclure. 
    """
    
    string =" ".join([mot for mot in string.lower().split(" ") if mot not in stopWords])
    return string

# créer une fonction pour stemmiser les mots d'un text
def stemmatise_text(text, stemmer):

    """
    Stemmatise un texte : Ramène les mots d'un texte à leur racine (peut créer des mots qui n'existe pas).

    Paramètres
    ----------

    text : Chaine de caractères.

    stemmer : Stemmer de NLTK.
    """
    
    ## code 
    string = " ".join([stemmer.stem(mot) for mot in text.lower().split(" ")])
    return string


In [22]:
def stem_cleaner(pandasSeries, stemmer, stopWords):
    
    print("#### Nettoyage en cours ####") # Mettre des print vous permet de comprendre où votre code rencontre des problèmes en cas de bug
    
    # confirmation que chaque document est bien de type str
    pandasSeries = pandasSeries.apply(lambda x : str(x))
    
    ### COMMENCEZ A CODER ICI! remplacer les 'None' par votre code ###
    
    # Passage en minuscule
    print("... Passage en minuscule") 
    pandasSeries = pandasSeries.apply(lambda x : x.lower())
    
    # Suppression des accents
    print("... Suppression des accents") 
    pandasSeries = pandasSeries.apply(lambda x : unidecode(x))
    
    # Détection du champs année, c'est pour l'exmple, dans les articles de sports les années peuvent avoir leur importance
    # Encore une fois ça se test, tout comme toutes les transformations envisageables
    print("... Détection du champs année") 
    pandasSeries = pandasSeries.apply(lambda x : re.sub(r'[0-9]{4}', 'annee', x))
    
    # Suppression des caractères spéciaux et numériques
    print("... Suppression des caractères spéciaux et numériques") 
    pandasSeries = pandasSeries.apply(lambda x :re.sub(r"[^a-z]+", ' ', x))
    
    # Suppression des stop words
    print("... Suppression des stop words") 
    pandasSeries = pandasSeries.apply(lambda x : no_stop_word(x, stopWords))
    
    # Stemmatisation
    print("... Stemmatisation")
    pandasSeries = pandasSeries.apply(lambda x : stemmatise_text(x, stemmer))
    
    print("#### Nettoyage OK! ####")

    return pandasSeries

In [23]:
%%time
overview['overview_stem'] = stem_cleaner(overview['overview'], stemmer, stopWords)

overview[['overview', 'overview_stem']].head()

#### Nettoyage en cours ####
... Passage en minuscule
... Suppression des accents
... Détection du champs année
... Suppression des caractères spéciaux et numériques
... Suppression des stop words
... Stemmatisation
#### Nettoyage OK! ####
CPU times: user 1.68 s, sys: 8.53 ms, total: 1.69 s
Wall time: 1.69 s


,overview,overview_stem
15480,"Cobb, a skilled thief who commits corporate es...",cobb skill thief commit corpor espionag infilt...
12481,Batman raises the stakes in his war on crime. ...,batman rais stake war crime help lt jim gordon...
22879,Interstellar chronicles the adventures of a gr...,interstellar chronicl adventur group explor ma...
17818,When an unexpected enemy emerges and threatens...,unexpect enemi emerg threaten global safeti se...
14551,"In the 22nd century, a paraplegic Marine is di...",nd centuri parapleg marin dispatch moon pandor...


Avec cette matrice en main, vous pouvez maintenant calculer un score de similarité. Il existe plusieurs mesures de similarité que vous pouvez utiliser pour cela, telles que les scores de similitude manhattan, euclidien, Pearson et cosinus. Encore une fois, il n’y a pas de bonne réponse à la question de savoir quel score est le meilleur. Différents scores fonctionnent bien dans différents scénarios, et c’est souvent une bonne idée d’expérimenter avec différentes mesures et d’observer les résultats.

Vous utiliserez le pour calculer une quantité numérique qui indique la similitude entre deux films. Vous utilisez le score de similarité cosinus car il est indépendant de l’ampleur et est relativement facile et rapide à calculer (surtout lorsqu’il est utilisé en conjonction avec les scores TF-IDF, qui seront expliqués plus tard). Mathématiquement, il est défini comme suit:cosine similarity


Puisque vous avez utilisé le vectoriseur TF-IDF, le calcul du produit ponctuel entre chaque vecteur vous donnera directement le score de similarité cosinus. Par conséquent, vous utiliserez à la place de car il est plus rapide.sklearn's linear_kernel()cosine_similarities()

Cela renverrait une matrice de forme 45466x45466, ce qui signifie que chaque film score de similitude cosinus avec tous les autres films. Par conséquent, chaque film sera un vecteur colonne 1x45466 où chaque colonne sera un score de similarité avec chaque film.overviewoverview

## 2_2 Compute the cosine similarity matrix

In [24]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

In [25]:
overview.reset_index(drop=True,inplace=True)

In [26]:
overview.head()

,overview,title,score,overview_stem
0,"Cobb, a skilled thief who commits corporate es...",Inception,683.559864,cobb skill thief commit corpor espionag infilt...
1,Batman raises the stakes in his war on crime. ...,The Dark Knight,610.407684,batman rais stake war crime help lt jim gordon...
2,Interstellar chronicles the adventures of a gr...,Interstellar,544.399840,interstellar chronicl adventur group explor ma...
3,When an unexpected enemy emerges and threatens...,The Avengers,535.835801,unexpect enemi emerg threaten global safeti se...
4,"In the 22nd century, a paraplegic Marine is di...",Avatar,527.031777,nd centuri parapleg marin dispatch moon pandor...


In [27]:
tfidf = TfidfVectorizer(ngram_range=(1, 3)) 
tfidf.fit(overview.loc[:,('overview_stem')].values.astype('U')) # Les parenthèses permettent d'extraire la colonne sous forme de série pandas. Ne rien mettre passait aussi mais bof…

overview_tfidf = tfidf.transform(overview['overview_stem'].values.astype('U')).toarray()

print(f"La base contient {overview_tfidf.shape[0]} obs et {overview_tfidf.shape[1]} features")

La base contient 4555 obs et 242681 features


In [28]:
cos_sim_overviews=linear_kernel(overview_tfidf)

In [29]:
cos_sim_overviews.shape

(4555, 4555)

Vous allez définir une fonction qui prend un titre de film en entrée et sort une liste des 10 films les plus similaires. Tout d’abord, pour cela, vous avez besoin d’un mappage inverse des titres de films et des index DataFrame. En d’autres termes, vous avez besoin d’un mécanisme pour identifier l’index d’un film dans votre DataFrame, compte tenu de son titre.metadata

## 2_3 Construct a reverse map of indices and movie titles


In [30]:
def reverse_map(title:str)->str:
    return (overview.index[overview["title"]==title][0])

Vous êtes maintenant en bonne forme pour définir votre fonction de recommandation. Voici les étapes suivantes que vous suivrez :

<li>Obtenez l’index du film étant donné son titre.

<li>Obtenez la liste des scores de similitude cosinus pour ce film particulier avec tous les films. Convertissez-le en une liste de tuples où le premier élément est sa position, et le second est le score de similarité.

<li>Triez la liste de tuples susmentionnée en fonction des scores de similarité; c’est-à-dire le deuxième élément.

<li>Obtenez les 10 principaux éléments de cette liste. Ignorez le premier élément car il se réfère à soi (le film le plus similaire à un film particulier est le film lui-même).

<li>Renvoyer les titres correspondant aux index des éléments supérieurs.

In [31]:
def liste_recommandations(title:str)->list:
    cos_sim_title=pd.Series(cos_sim_overviews[reverse_map(title),:])
    top_cos_sims=cos_sim_title.sort_values(ascending=False).head(11)[1:]
    top_indexes=top_cos_sims.index.tolist()
    liste=overview.loc[top_indexes,"title"].tolist()
    return liste

In [153]:
liste_recommandations("Inception")

['Cypher',
 'What Ever Happened to Baby Jane?',
 'Mission: Impossible - Rogue Nation',
 'Minority Report',
 'The East',
 'Descendants',
 'Ghost Dog: The Way of the Samurai',
 'Much Ado About Nothing',
 'Central Intelligence',
 'The Net']

# 3 Recommandation  basé sur les crédits, les genres et les mots-clés

La qualité de votre recommandateur serait améliorée avec l’utilisation de meilleures métadonnées et en capturant plus de détails plus fins. C’est précisément ce que vous allez faire dans cette section. Vous construirez un système de recommandation basé sur les métadonnées suivantes: les 3 meilleurs acteurs, le réalisateur, les genres associés et les mots-clés de l’intrigue du film.

Les mots-clés, les données de distribution et d’équipe ne sont pas disponibles dans votre jeu de données actuel, la première étape consiste donc à les charger et à les fusionner dans votre DataFrame principal.metadata

In [35]:
# Load keywords and credits
credits = pd.read_csv("data/credits.csv")
keywords = pd.read_csv("data/keywords.csv")

# Remove rows with bad IDs.
mov_met = mov_met.drop([19730, 29503, 35587])

# Convert IDs to int. required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
mov_met['id'] = mov_met['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
mov_met = mov_met.merge(credits, on='id')
mov_met = mov_met.merge(keywords, on='id')

In [36]:
mov_met.iloc[[19730, 29503, 35587],:]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y
19730,False,NaN,0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://www.studio-rikka.com/page/pale/pale_top...,20683,tt1506448,ja,ペイル・コクーン,In the far future when the continuity of the h...,...,Pale Cocoon,False,6.8,16.0,"[{'cast_id': 2, 'character': 'Yoko Yamaguchi',...","[{'credit_id': '52fe43f3c3a368484e007aa1', 'de...","[{'id': 4458, 'name': 'post-apocalyptic'}, {'i...","[{'cast_id': 2, 'character': 'Yoko Yamaguchi',...","[{'credit_id': '52fe43f3c3a368484e007aa1', 'de...","[{'id': 4458, 'name': 'post-apocalyptic'}, {'i..."
29503,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://www.mylifetime.com/movies/secrets-of-eden,88318,tt2191805,en,Secrets of Eden,Based on Chris Bohjalian’s New York Times best...,...,Secrets of Eden,False,4.8,6.0,"[{'cast_id': 4, 'character': 'Catherine Benica...","[{'credit_id': '52fe49f09251416c910bdd1d', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 4, 'character': 'Catherine Benica...","[{'credit_id': '52fe49f09251416c910bdd1d', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
35587,False,NaN,0,[],http://www.imdb.com/title/tt0232079/,159666,tt0232079,en,Mela,"Roopa, a beautiful young village belle is much...",...,Mela,False,5.5,4.0,"[{'cast_id': 2, 'character': 'Kishan Pyare', '...","[{'credit_id': '52fe4c1a9251416c910efd97', 'de...",[],"[{'cast_id': 2, 'character': 'Kishan Pyare', '...","[{'credit_id': '52fe4c1a9251416c910efd97', 'de...",[]


À partir de vos nouveaux longs métrages, acteurs, équipes et mots-clés, vous devez extraire les trois acteurs les plus importants, le réalisateur et les mots-clés associés à ce film.

Mais tout d’abord, vos données sont présentes sous la forme de listes « stringifiées ». Vous devez les convertir d’une manière utilisable pour vous.

In [25]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:

    metadata[feature] = metadata[feature].apply(literal_eval)


Ensuite, vous écrivez des fonctions qui vous aideront à extraire les informations requises de chaque fonctionnalité.

Tout d’abord, vous allez importer le paquet NumPy pour accéder à sa constante. Ensuite, vous pouvez l’utiliser pour écrire la fonction:NaNget_director()

In [26]:
# Import Numpy
import numpy as np

Obtenez le nom du réalisateur dans la fonction d’équipe. Si le directeur ne figure pas dans la liste, retournez NaN

Ensuite, vous allez écrire une fonction qui retournera les 3 premiers éléments ou la liste entière, selon le montant le plus élevé. Ici, la liste fait référence aux , et .castkeywordsgenres

Définir de nouvelles caractéristiques de directeur, de distribution, de genres et de mots-clés qui sont dans une forme appropriée.

In [49]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [ ]:
Affichier les nouveautés des 3 premiers films

L’étape suivante consisterait à convertir les noms et les occurrences de mots-clés en minuscules et à supprimer tous les espaces entre eux.

La suppression des espaces entre les mots est une étape de prétraitement importante. C’est fait pour que votre vectorisateur ne compte pas le Johnny de « Johnny Depp » et « Johnny Galecki » comme identique. Après cette étape de traitement, les acteurs susmentionnés seront représentés comme « johnnydepp » et « johnnygalecki » et seront distincts de votre vectoriseur.

Un autre bon exemple où le modèle pourrait produire la même représentation vectorielle est « embouteillage » et « embouteillage ». Par conséquent, il est préférable de dépouiller tout espace présent.

Ecrire une fonction qui convertir toutes les chaînes en minuscules et les noms de bandes d’espaces

Appliquer la fonction  à vos fonctionnalités. 'cast', 'keywords', 'director', 'genres'

Vous êtes maintenant en mesure de créer votre « soupe de métadonnées », qui est une chaîne qui contient toutes les métadonnées que vous souhaitez alimenter votre vectoriseur (à savoir les acteurs, le réalisateur et les mots-clés).

La fonction joindra simplement toutes les colonnes requises par un espace. Il s’agit de la dernière étape de prétraitement, et la sortie de cette fonction sera introduite dans le modèle vectoriel verbal.create_soup